# Simulated Annealing for Feature Selection
## 03 - Simulated Annealing Algorithm (Experimentation)
References
- https://stackoverflow.com/questions/37105696/how-to-have-a-set-of-sets-in-python
___

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
import random

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Define path
PATH = '../data/'

# Set seed
SEED = 42

In [4]:
X_train = pd.read_csv(PATH + 'processed/X_train.csv')
y_train = pd.read_csv(PATH + 'processed/y_train.csv')

In [10]:
np.arange(len(X_train.columns))

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [21]:
full_set = set(np.arange(len(X_train.columns)))
full_set

{0, 1, 2, 3, 4, 5, 6, 7, 8}

In [139]:
# Generate random initial subset and filter df by column indices
curr_subset = set(random.sample(full_set, 4))
curr_subset

{0, 1, 2, 8}

In [144]:
round(0.5 * len(X_train.columns))

4

In [140]:
X_train.iloc[:, list(curr_subset)]

,Pclass,Sex,Age,Title
0,3,0,22.0,2
1,1,1,38.0,3
2,3,1,26.0,1
3,1,1,35.0,3
4,3,0,35.0,2
...,...,...,...,...
886,2,0,27.0,4
887,1,1,19.0,1
888,3,1,13.5,1
889,1,0,26.0,2


In [122]:
# Decide what to do first
if len(curr_subset) == len(full_set):
    move = random.choice(['Remove'])
elif len(curr_subset) == 2:
    move = random.choice(['Add', 'Replace'])
else:
    move = random.choice(['Add', 'Replace', 'Remove'])

move

'Add'

In [134]:
hash_values = set()
hash_values.add(frozenset([1,2]))
hash_values.add(frozenset([3,4]))

In [86]:
# Get difference in sets
pending_cols = full_set.difference(curr_subset)
pending_cols

{2, 4, 6, 7, 8}

In [109]:
# assert len(new_subset) - len(curr_subset) == 1, "New subset does not have +1 feature count compared to current subset"

In [135]:
# Perturb
while True:  
    pending_cols = full_set.difference(curr_subset)
    new_subset = curr_subset.copy()
    
    if move == 'Add':        
        new_subset.add(random.choice(list(pending_cols)))
    elif move == 'Replace': 
        new_subset.remove(random.choice(list(curr_subset)))
        new_subset.add(random.choice(list(pending_cols)))
    else:
        new_subset.remove(random.choice(list(curr_subset)))
        
    if new_subset in hash_values:
        'Subset already visited'
    else:
        hash_values.add(frozenset(new_subset)) # !!!! Review whether include here or after modeling !!!
        break

In [136]:
hash_values

{frozenset({3, 4}), frozenset({1, 2}), frozenset({0, 1, 2, 3, 5})}

___

In [145]:
def train_model(X, y):
    """
    Run random forest classification model on feature subset
    and retrieve cross validated ROC-AUC score
    """
    clf = RandomForestClassifier(random_state=SEED)
    kf = KFold(shuffle=True, n_splits=3, random_state=SEED)
    cv_roc_auc_score = round(cross_val_score(clf, X, y, cv=kf, 
                                             scoring="roc_auc", n_jobs=-1).mean(), 3)

    return cv_roc_auc_score


In [146]:
def simulate_annealing(X_train,
                       Y_train,
                       maxiters=10,
                       alpha=0.85,
                       beta=1.3,
                       T_0=0.95,
                       update_iters=1):
    """
    Function to perform feature selection using simulated annealing
    Inputs:
    X_train - Train Predictor Features
    Y_train - Train Labels
    maxiters - Maximum number of iterations
    alpha - factor to reduce temperature
    # beta - constant in probability estimate 
    T_0 - Initial temperature
    update_iters - Number of iterations required to update temperature

    Output:
    1) Dataframe of the parameters explored and corresponding model performance
    2) Column indices of selected features
    """
    columns = ['Iteration', 'Feature Count', 'Current AUC', 'New AUC', 
               'Acceptance Probability', 'Random Number', 'Outcome']
    results = pd.DataFrame(index=range(maxiters), columns=columns)
    best_metric = -1.
    hash_values = set()
    T = T_0

    # Get ascending range indices of all columns
    full_set = set(np.arange(len(X_train.columns)))

    # Generate initial random subset based on ~50% of columns
    curr_subset = set(random.sample(full_set, round(0.5 * len(full_set))))

    # Get baseline AUC score of initial random subset
    X_curr = X_train.iloc[:, list(curr_subset)]
    prev_metric = train_model(X_curr, y_train)
    
    return prev_metric

In [147]:
simulate_annealing(X_train, y_train)

0.851

___

In [201]:
def simulated_annealing(X_train,
                        y_train,
                        maxiters=50,
                        alpha=0.85,
                        beta=1,
                        T_0=0.95,
                        update_iters=1,
                        temp_reduction='geometric'):
    """
    Function to perform feature selection using simulated annealing
    Inputs:
    X_train - Train Predictor Features
    y_train - Train Labels
    maxiters - Maximum number of iterations
    alpha - factor to reduce temperature
    # beta - constant in probability estimate 
    T_0 - Initial temperature
    update_iters - Number of iterations required to update temperature

    Output:
    1) Dataframe of the parameters explored and corresponding model performance
    2) Column indices of selected features
    """
    columns = ['Iteration', 'Feature Count', 'Feature Set', 
               'Metric', 'Best Metric', 'Acceptance Probability', 
               'Random Number', 'Outcome']
    results = pd.DataFrame(index=range(maxiters), columns=columns)
    best_subset = None
    hash_values = set()
    T = T_0

    # Get ascending range indices of all columns
    full_set = set(np.arange(len(X_train.columns)))

    # Generate initial random subset based on ~50% of columns
    curr_subset = set(random.sample(full_set, round(0.5 * len(full_set))))

    # Get baseline metric score (i.e. AUC) of initial random subset
    X_curr = X_train.iloc[:, list(curr_subset)]
    prev_metric = train_model(X_curr, y_train)
    best_metric = prev_metric

    for i in range(maxiters):
        print(T)
        # Termination conditions
        if T < 0.05:
            print(f'Temperature {T} below threshold. Termination condition met')
            break
        
        print(f'Starting Iteration {i}')
        print(curr_subset)
        print(prev_metric)
        # Decide what type of pertubation to make
        if len(curr_subset) == len(full_set): 
            move = 'Remove'
        elif len(curr_subset) == 2: # Not to go below 2 features
            move = random.choice(['Add', 'Replace'])
        else:
            move = random.choice(['Add', 'Replace', 'Remove'])

        # Execute pertubation (i.e. alter current subset to get new subset)
        while True:
            # Get columns not yet used in current subset
            pending_cols = full_set.difference(curr_subset) 
            new_subset = curr_subset.copy()   

            if move == 'Add':        
                new_subset.add(random.choice(list(pending_cols)))
            elif move == 'Replace': 
                new_subset.remove(random.choice(list(curr_subset)))
                new_subset.add(random.choice(list(pending_cols)))
            else:
                new_subset.remove(random.choice(list(curr_subset)))
                
            if new_subset in hash_values:
                'Subset already visited'
            else:
                hash_values.add(frozenset(new_subset)) # !!!! Review whether include here or after modeling !!!
                break

        # Filter dataframe to current subset
        X_new = X_train.iloc[:, list(new_subset)]

        # Get metric of new subset
        metric = train_model(X_new, y_train)

        if metric > prev_metric:
            print('Local improvement in metric from {:8.4f} to {:8.4f} '
                  .format(prev_metric, metric) + ' - parameters accepted')
            outcome = 'Improved'
            accept_prob, rnd = '-', '-'
            prev_metric = metric
            curr_subset = new_subset.copy()

            # Keep track of overall best metric so far
            if metric > best_metric:
                print('Global improvement in metric from {:8.4f} to {:8.4f} '
                      .format(best_metric, metric) + ' - best parameters updated')
                best_metric = metric
                best_subset = new_subset.copy()
                
        else:
            rnd = np.random.uniform()
            diff = metric - prev_metric
            accept_prob = np.exp(beta * diff / T)

            if rnd < accept_prob:
                print('New subset has worse performance but still accept. Metric change' +
                      ': {:8.4f} acceptance probability: {:6.4f} random number: {:6.4f}'
                      .format(diff, accept_prob, rnd))
                outcome = 'Accept'
                prev_metric = metric
                curr_subset = new_subset.copy()
            else:
                print('New subset has worse performance, therefore reject. Metric change' +
                      ': {:8.4f} acceptance probability: {:6.4f} random number: {:6.4f}'
                      .format(diff, accept_prob, rnd))
                outcome = 'Reject'

        # Update results dataframe
        results.loc[i, 'Iteration'] = i
        results.loc[i, 'Feature Count'] = len(curr_subset)
        results.loc[i, 'Feature Set'] = list(curr_subset)
        results.loc[i, 'Metric'] = metric
        results.loc[i, 'Best Metric'] = best_metric
        results.loc[i, 'Acceptance Probability'] = accept_prob
        results.loc[i, 'Random Number'] = rnd
        results.loc[i, 'Outcome'] = outcome

        # Temperature cooling schedule
        if i % update_iters == 0:
            if temp_reduction == 'geometric':
                T = alpha * T
            elif temp_reduction == 'linear':
                T -= alpha
            elif temp_reduction == 'slow decrease':
                b = 5 # Arbitrary constant
                T = T / (1 + b * T)
            else:
                raise Exception("Temperature reduction strategy not recognized")

    # Drop NaN rows in results
#     results = results.dropna(axis=0, how='all')

    # Convert column indices of best subset to original names
    best_subset_cols = [list(X_train.columns)[i] for i in list(best_subset)]

    return results, best_metric, best_subset_cols

In [202]:
results, best_metric, best_subset_cols = simulate_annealing(X_train, y_train)

0.95
Starting Iteration 0
Local improvement in metric from   0.7020 to   0.7320  - parameters accepted
Global improvement in metric from   0.7020 to   0.7320  - best parameters updated
0.8075
Starting Iteration 1
New subset has worse performance but still accept. Metric change:  -0.0120 acceptance probability: 0.9852 random number: 0.0968
0.686375
Starting Iteration 2
New subset has worse performance but still accept. Metric change:  -0.0280 acceptance probability: 0.9600 random number: 0.1132
0.58341875
Starting Iteration 3
Local improvement in metric from   0.6920 to   0.7030  - parameters accepted
0.49590593749999995
Starting Iteration 4
New subset has worse performance but still accept. Metric change:  -0.0940 acceptance probability: 0.8273 random number: 0.3956
0.42152004687499994
Starting Iteration 5
Local improvement in metric from   0.6090 to   0.7760  - parameters accepted
Global improvement in metric from   0.7320 to   0.7760  - best parameters updated
0.35829203984374997
Sta

In [203]:
results

,Iteration,Feature Count,Metric,Best Metric,Acceptance Probability,Random Number,Outcome
0,0,4,0.732,0.732,-,-,Improved
1,1,3,0.72,0.732,0.985249,0.096769,Accept
2,2,2,0.692,0.732,0.960027,0.113193,Accept
3,3,2,0.703,0.732,-,-,Improved
4,4,2,0.609,0.732,0.82733,0.395611,Accept
5,5,2,0.776,0.776,-,-,Improved
6,6,2,0.804,0.804,-,-,Improved
7,7,3,0.833,0.833,-,-,Improved
8,8,2,0.718,0.833,0.641308,0.58304,Accept
9,9,3,0.696,0.833,0.904852,0.151263,Accept


In [175]:
test = X_train.columns.tolist()
test

['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'FamilySize',
 'Title']

In [182]:
list(X_train.columns)

'SibSp'

In [179]:
res_list = [test[i] for i in [1,3,5]]
res_list

['Sex', 'SibSp', 'Fare']

In [171]:
results

,Iteration,Feature Count,Metric,Best Metric,Acceptance Probability,Random Number,Outcome
0,0,4,0.851,0.851,-,-,Improved
1,1,4,0.695,0.851,0.824326,0.621874,Accept
2,2,4,0.61,0.851,0.883522,0.238862,Accept
3,3,4,0.617,0.851,-,-,Improved
4,4,3,0.615,0.851,0.995975,0.541772,Accept
5,5,3,0.782,0.851,-,-,Improved
6,6,3,0.843,0.851,-,-,Improved
7,7,4,0.833,0.851,0.967698,0.542027,Accept
8,8,4,0.683,0.851,0.560206,0.131384,Accept
9,9,3,0.68,0.851,0.986458,0.462377,Accept
